## 📥 Fetch data

We'll fetch contents from VS Code docs ([GitHub repo](https://github.com/microsoft/vscode-docs/blob/main)) using the GitHub API and save it into a json format that can be used for RAG models.

In [1]:
# Recursively fetch all md files from a GH repo using gh api

import os
import requests
from dotenv import load_dotenv

load_dotenv()

REPO_OWNER = 'microsoft'
REPO_NAME = 'vscode-docs'

def get_markdown_files_from_github(path=''):
    url = f'https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{path}'
    headers = {
        'Authorization': f'token {os.getenv("GITHUB_TOKEN")}'
    }
    response = requests.get(url, headers=headers)
    markdown_files = []

    if response.status_code == 200:
        files = response.json()

        for file in files:
            if file['type'] == 'dir':
                markdown_files += get_markdown_files_from_github(file['path'])
            elif file['type'] == 'file' and file['name'].endswith('.md'):
                markdown_files.append(file['path'])

        return markdown_files
    else:
        raise Exception(f"Error fetching repo contents: {response.status_code}")
    
# Test: Print all markdown files in the learn/students folder
print(get_markdown_files_from_github("learn/students"))

['learn/students/github-pack.md', 'learn/students/nasa-python.md', 'learn/students/over-the-moon-python.md', 'learn/students/spacejam-python.md', 'learn/students/wonder-woman-python.md']


In [2]:
# Get the content of a file using gh api

import base64

def get_file_content(file_path):
    url = f'https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{file_path}'
    headers = {
        'Authorization': f'token {os.getenv("GITHUB_TOKEN")}'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        file_content = base64.b64decode(response.json()['content']).decode('utf-8')
        title = next(line.replace('# ', '') for line in file_content.split('\n') if line.startswith('#'))
        description = next((line.split(':', 1)[1].strip() for line in file_content.split('\n') if line.lower().startswith('metadescription:')), '')
        content = file_content.split(title, 1)[1].strip()
        content = content.replace('<!-- DOWNLOAD_LINKS_PLACEHOLDER -->', '')
        full_content = f"{description}\n\n{content}"
        return {
            'content': full_content, # rate limit could be exceeded for free tier
            'url': f'https://github.com/{REPO_OWNER}/{REPO_NAME}/blob/main/{file_path}',
        }
    else:
        raise Exception(f"Error fetching file content: {response.status_code}")
    
# Test: Print a sample file content
get_file_content("release-notes/v1_94.md")

{'content': 'Learn what is new in the Visual Studio Code September 2024 Release (1.94)\n\n**Update 1.94.1**: The update addresses this security [issue](https://github.com/microsoft/vscode/issues?q=is%3Aissue+is%3Aclosed+milestone%3A%22September+2024+Recovery+1%22+).\n\n**Update 1.94.2**: The update addresses these [issues](https://github.com/microsoft/vscode/issues?q=is%3Aissue+milestone%3A%22September+2024+Recovery+2%22+is%3Aclosed).\n\n\n\n---\n\nWelcome to the September 2024 release of Visual Studio Code. There are many updates in this version that we hope you\'ll like, some of the key highlights include:\n\n* [Find in Explorer](#find-in-explorer) - Quickly find files in the Explorer view with the improved Find control.\n* [Source Control Graph](#source-control) - More filtering options and interactivity in the Source Control Graph.\n* [Python test coverage](#python) - Run Python tests with coverage and get rich results in the editor.\n* [ESM](#esm-is-shipping-for-vs-code) - Faster 

In [3]:
# Iterate over each file in `release-notes` and get the content

docs_files = get_markdown_files_from_github("release-notes")

docs_contents = []
for file_path in docs_files:
    content = get_file_content(file_path)
    docs_contents.append(content)

# Print the first few contents to verify
for doc in docs_contents[:5]:
    print(doc)
    print('---')

{'content': 'See what is new in the Visual Studio Code July 2016 Release (1.4)\n\nDuring July, we slowed down feature work in favor of reducing our bug backlog and removing engineering debt. However, we were still able to add some improvements.\n\nHere are the highlights:\n\n* **Workbench**: Editor actions such as **Open Preview** and **Switch to Changes View** are back on the title bar. IME and Copy/Paste support in the Integrated Terminal.\n* **Editor**: Better snippet and suggestions control. New **Insert Snippet** command with dedicated UI.\n* **Debugging**: **Restart Frame** action to rerun a specific stack frame.  \'Variable paging\' feature moved into VS Code and available to all debug extensions.\n* **Extension Authoring**: New \'move\' commands to better support VIM gestures. Custom link behavior with the `DocumentLinkProvider` API. Expanded Debug Protocol.\n\nDownloads: [Windows](https://az764295.vo.msecnd.net/stable/6276dcb0ae497766056b4c09ea75be1d76a8b679/VSCodeSetup-stable

## 🧩 Chunk data

In [4]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3")
]

In [8]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re

# Initialize the MarkdownHeaderTextSplitter with the headers to split on
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

# Split each document content and retain the URL
split_docs = []
for doc in docs_contents:
    split_content = markdown_splitter.split_text(doc['content'])
    first_line = doc['content'].split('\n', 1)[0]
    for split in split_content:
        if 'Header 3' in split.metadata:
            header_text = split.metadata['Header 3'].replace(' ', '-').lower()
            new_url = re.sub(r'https://github.com/microsoft/vscode-docs/blob/main/release-notes/(.*)\.md', r'https://code.visualstudio.com/updates/\1', doc['url'])
            split_docs.append({
                'content': f"{first_line}\n\n{split.page_content}",
                'url': f"{new_url}#{header_text}",
            })

# Print the first few split documents to verify
for split_doc in split_docs[:5]:
    print(split_doc)
    print('---')

{'content': 'See what is new in the Visual Studio Code July 2016 Release (1.4)\n\nBy default, VS Code shows snippets and completion proposals in one widget. This is not always desirable so we added a new configuration setting called `editor.snippetSuggestions`. To remove snippets from the suggestions widget, set the value to `"none"`. If you\'d like to see snippets, you can specify the order relative to suggestions; at the top (`"top"`), at the bottom (`"bottom`"), or inline ordered alphabetically (`"inline"`). The default is `"inline"`.  \nIn addition, we added a new command to insert a snippet, **Insert Snippet**, which displays the available snippets in a dropdown.  \n![snippet insert](images/July_2016/insertSnippet.gif)', 'url': 'https://code.visualstudio.com/updates/July_2016#snippets-and-suggestions'}
---
{'content': 'See what is new in the Visual Studio Code July 2016 Release (1.4)\n\nThe editor now supports `kbstyle(Tab)` completions. You can enable it with the `editor.tabCompl

### 📄 Output as Json

In [9]:
import json

# Write split_docs_dicts to a JSON file
with open('split_docs_contents.json', 'w') as f:
    json.dump(split_docs, f, indent=4)